# Importing Libraries

In [30]:
import pandas as pd
import numpy as np
import random
from copy import deepcopy
from itertools import combinations

# Importing Data

In [31]:
teachers = pd.read_csv('Data/teachers.csv', header=None)
courses = pd.read_csv('Data/courses.csv', header=None)
studentCourse = pd.read_csv('Data/studentCourse.csv', index_col=0)
studentNames = pd.read_csv('Data/studentNames.csv', header=None)

teacherslist = list(set(teachers[0]))[0:10]
courseslist = list(set(courses[0]))

In [32]:
#Filtering MG Courses from CoursesList
MGcourse = []
for course in courseslist:
    if 'MG' in course:
        MGcourse.append(course)
        
#MGcourse

In [33]:
#Optional Courses and Student to Remove
#courseslist.remove('EE227')
#courseslist.remove('CY2012')
#studentCourse[studentCourse['Student Name'] == "Syed M Ghufran" ]

#For Answer in 9 days, Else Ans come in 2 Weeks
studentCourse.drop(studentCourse[studentCourse['Student Name'] == 'Syed M Ghufran'].index,inplace=True)
studentCourse.drop(studentCourse[studentCourse['Student Name'] == 'Sarah E Kirkham-Slack'].index,inplace=True)

In [34]:
#It Calculates and Stores All Students Clashes in a Dictionary
clashDict = {}

for all in courseslist:
    l = set(studentCourse[studentCourse['Course Code'] == all]['Student Name'])
    clashDict[all] = l
    
#clashDict

# DF Queries

## Number of Enrolled Courses Per Student

In [35]:
studentCourse.groupby(['Student Name']).count()

,Course Code
Student Name,
Abdul Gafur,5
Adam C Ryan,4
Adam Flowers,4
Adam Green,4
Adam Kirk,4
...,...
Zahir Hussain,5
Zahra Faraji Rad,5
Zainab Khattak,4


## Number of Students Per Course

In [36]:
studentCourse.groupby(['Course Code']).count()

,Student Name
Course Code,
AI2011,46
CS118,28
CS211,56
CS217,29
CS218,35
CS219,23
CS220,25
CS302,27
CS307,40


## Get List of Students Clashes in 2 Courses

In [37]:
def get_clashes(course1, course2):
   
    return clashDict[course1].intersection(clashDict[course2])
    
#get_clashes('MG220', 'MG223')

## Number of Students in a Particular Course

In [38]:
#l is course code
def get_num_of_students(L):
    return len(clashDict[L])

#get_num_of_students('MG220')

# Classes

In [39]:
class Exam:
    
    def __init__(self,day,slot,course,teacher):
        self.day = day
        self.slot = slot
        self.course = course
        self.teacher =teacher
    
    def __repr__(self):
        s = {"Course ": self.course,"Day" : self.day,"Slot" : self.slot,"Teachr" : self.teacher}
        return str(s)
    
    def __eq__(self,other):
        if self.day == other.day :
            if self.slot == other.slot :
                if self.course == other.course :
                    return True
        
        
        return False
    
    def __hash__(self):
        return hash(self.day) ^ hash(self.slot) ^ hash(self.course) 
        
    def to_dict(self):
        s = {"Course ": self.course,"Day" : self.day,"Slot" : self.slot,"Teachr" : self.teacher}
        return s
    

# A function that returns the 'Day' and 'Slot' of a Particular Exam
def retDaySlot(e):
  return e.day, e.slot

# Functions


## Generate Random Schedule

In [40]:
def Generate_Random_Schedule(tl, cl, numSlots = 2, maxStudentsPerSlot = 70 ):

    teacherList = [t for t in tl]
    coursesList = [t for t in cl]            
    
    _schedule = []
    
    
    
    #max number of exams to conduct in each slot
    #maxperslot = 3
    
    
    for index, value in enumerate(weekDays):
        if len(coursesList) == 0:
            break

        # Generate n Slots for A Day
        
        slots = []
        for i in range(numSlots):

            if len(coursesList) == 0:
                break

            # Generate Exams For One Slots
            exams = []
            numStd = 0
            
            maxcounter = 0
            while(numStd <= maxStudentsPerSlot):
                
                 
                if len(coursesList) == 0:
                    break

                if(numStd > maxStudentsPerSlot):
                    break
                    
                randomIndex = random.randrange(0, len(coursesList))
                randomCourse = coursesList.pop(randomIndex)

                randomIndex = random.randrange(0, len(teacherList))
                randomTeacher = teacherList[randomIndex]

                numStd += get_num_of_students(randomCourse)
                
                maxcounter+=1
                
                   
                _schedule.append(Exam(value,i,randomCourse, randomTeacher))
 
    return _schedule


## Generate Population

In [41]:
def generate_random_population(size = 100, seats = 70):
    return [Generate_Random_Schedule(teacherslist,courseslist,maxStudentsPerSlot=seats) for _ in range(size)]

#Scds = generate_random_population(10)
#Scds

## Fitness Function

In [42]:
def Fitness(schedule):

    combos = set([(row.day, row.slot) for row in schedule])

    softFitness = 0
    totalFitness = 0

    # Hard Constraint Fitness
    for session in combos:

        coursenames = [c.course for c in schedule if (c.day, c.slot) == session]
        
    
        combos = combinations(coursenames, 2)

        for clash in combos:

            clashlen = len(get_clashes(clash[0], clash[1]))

            if clashlen > 0:
                numofstudentsclashing = clashlen
                totalFitness += numofstudentsclashing

                
    # if Hardconstraint fit then set solved to True
    solved = False
    if totalFitness == 0:
        solved = True
        schedule.sort(key=retDaySlot)
        # no need to calculate rest since hard constraint complete

        
    # Soft Constraint Calculation

        # All students and teachers shall be given a break on Friday from 1-2.
        # A student shall not give more than 1 exam consecutively

    
    slot2courses = [c.course for c in schedule if (c.day == session[0] and c.slot != session[1])]
    for c1 in coursenames:
        for c2 in slot2courses:
            consec = len(get_clashes(c1, c2))

            if consec:
                softFitness += consec / 2

    # at leasthalf thefaculty is free in one slot and therest of thefaculty is free in the other slot so the faculty meetings shall be held in parts as they are now.
    if len(coursenames) > len(teacherslist)/2:
        softFitness += 2

    # management courses before rest
    unMatchCount = 0
    remainingCount = len(MGcourse)
    daySlot = 0 + (0 + 0)  # day + (day + slot)
    schedule.sort(key=retDaySlot)
    for row in schedule:
        if(remainingCount == 0) or (unMatchCount == len(MGcourse)):
            break
        elif(row.course in MGcourse):
            remainingCount -= 1
        if (row.day + (row.day + row.slot)) > daySlot:
            unMatchCount += 1
            daySlot = row.day + (row.day + row.slot)
    softFitness += remainingCount

    #print("before soft : ",totalFitness)
    totalFitness += softFitness

    return totalFitness, solved


def Fitness_with_print(schedule):

    combos = set([(row.day, row.slot) for row in schedule])

    softFitness = 0

    totalFitness = 0

    # Hard Constraint Fitness
    for session in combos:
        # only checking clash by student for now
        # todo : teacher count
        # student per session
        # clashing courses per day in slot

        coursenames = [c.course for c in schedule if (
            c.day, c.slot) == session]

        combos = combinations(coursenames, 2)

        for clash in combos:
            clasheslist = get_clashes(clash[0], clash[1])
            clashlen = len(clasheslist)

            if clashlen > 0:
                print("Hard Clashes : ",
                      clash[0], clash[1], session, list(clasheslist))
                numofstudentsclashing = clashlen
                totalFitness += numofstudentsclashing

    # if Hardconstraint fit then set solved to True
    solved = False
    if totalFitness == 0:
        solved = True
        # no need to calculate rest since hard constraint complete

    # Soft Constraint Calculation

    # A student shall not give more than 1 exam consecutively
    slot2courses = [c.course for c in schedule if (
        c.day == session[0] and c.slot != session[1])]

    for c1 in coursenames:
        for c2 in slot2courses:
            conseclist = get_clashes(c1, c2)
            consec = len(conseclist)

            if consec:
                print("Soft Clash : ", c1, c2, list(conseclist))
                softFitness += consec / 2

    # management courses before rest
    unMatchCount = 0
    remainingCount = len(MGcourse)
    daySlot = 0 + (0 + 0)  # day + (day + slot)
    schedule.sort(key=retDaySlot)
    for row in schedule:
        if(remainingCount == 0) or (unMatchCount == len(MGcourse)):
            break
        elif(row.course in MGcourse):
            remainingCount -= 1
        if (row.day + (row.day + row.slot)) > daySlot:
            unMatchCount += 1
            daySlot = row.day + (row.day + row.slot)
    softFitness += remainingCount

    if remainingCount != 0:
        print(f"{remainingCount} MG Courses Exams were Taken After CS Exams")
    
    #print("before soft : ",totalFitness)
    totalFitness += softFitness

    return totalFitness, solved

In [43]:
def Fitness_to_dictionary(listofpop):
    
    #population = [ {"Schedule": pop,"fitness" : Fitness(pop) } for pop in listofpop]
    
    population_wf = []
    
    for pop in listofpop:
        F,S = Fitness(pop)#F fitness , S Schedule
        
        SC = {"Schedule" : pop , "fitness":F , "solved":S}
        
        population_wf.append(SC)
            
    return population_wf


#population = generate_random_population(size = 1)
#Fitness_to_dictionary(population)


## Fixing Teacher Consecutive Invigilation (Hard Constraint)

In [44]:
def checkHardConstraint(schd):
    for day in weekDays:
        nTeacher = []
        indexOfSlots = []
        for index, one in enumerate(schd):
            #Selcting the Day
            if one.day == day:
                nTeacher.append(one.teacher)
                indexOfSlots.append(index)
        
        #Both Slots have Papers
        if (len(nTeacher) > 1):
            for i in range(len(nTeacher)-1):
                if nTeacher[i] == nTeacher[i+1]:
                    randTeacher = teacherslist[random.randrange(0, len(teacherslist))]
                    #Teacher is same, assign another teacher
                    while(nTeacher[i] == randTeacher):
                        randTeacher = teacherslist[random.randrange(0, len(teacherslist))]
                    #Replacing Teacher of that Slot
                    schd[indexOfSlots[i]].teacher = randTeacher

## Crossover Function

In [45]:
def crossover(parent1, parent2, cl = courseslist):

    #Shuffling the courses list
    coursesList = [c for c in cl]
    random.shuffle(coursesList)

    
    #INSTEAD OF CHOOSING RANDOM PROB, GENERATE A RANDOM THRESHOLD
    randomThreshold = random.randrange(0, len(coursesList))
    
    child = []
    
    #Selecting course from parent1
    for index in range(0, randomThreshold):
        for exam in parent1["Schedule"]:
            if exam.course == coursesList[index]:
                gene = exam
                break
        child.append(gene)
        
        
    #Selecting course from parent2
    for index in range(randomThreshold,len(coursesList)):
        for exam in parent2["Schedule"]:
            if exam.course == coursesList[index]:
                gene = exam
                break
        child.append(gene)
     
    
    return child

In [46]:
def n_crossover(parent1, parent2,n = 10):
    newchildren = []
    
    for i in range(n):
        child = (crossover(parent1,parent2))
        newchildren.append(child)
    
    return newchildren

## Generate Offsprings

In [47]:
def generate_offsprings(numtogenerate,Survivors):
 
    
    newchildren = []
        

    bf = int(numtogenerate / len(Survivors)-1) 
    
    
    for i in range (len(Survivors)-1):
            
        newchildren.extend(  n_crossover(Survivors[i],Survivors[i+1],bf) )
            
                
    remaining  = numtogenerate - len(newchildren)
    
    if remaining > 0 :    
        newchildren.extend(  n_crossover(Survivors[i],Survivors[i+1],remaining) )
        
    
    return newchildren
    

def generate_offsprings_with_roulette(numtogenerate,Survivors):
 
    #will produce 10 parents to be used from selected 
    p = np.array([ 1/(x['fitness']) for x in Survivors])
    p /= p.sum()#normalization 
    selectedparents = np.random.choice(Survivors,10, p = p)
    
    
    
    newchildren = []
    
    
    Survivors = selectedparents
    
    bf = int(numtogenerate / len(Survivors)-1) 
    
    
    for i in range (len(Survivors)-1):
            
        newchildren.extend(  n_crossover(Survivors[i],Survivors[i+1],bf) )
            
                
    remaining  = numtogenerate - len(newchildren)
    
    
    if remaining > 0 :    
        newchildren.extend(  n_crossover(Survivors[0],Survivors[1],remaining) )
        
    
    return newchildren

## Mutation Function

In [48]:
def mutate_child(schedule, changeby=1, cl=courseslist):

    skadoodle = [exam for exam in schedule]

    for i in range(changeby):
        course = random.choice(courseslist)

        for exams in skadoodle:

            if exams.course == course:
                skadoodle.remove(exams)

                # self,day,slot,course,teacher

                E = Exam(random.choice(weekDays), random.randint(
                    0, 1), course, exams.teacher)

                skadoodle.append(E)

    return skadoodle

In [49]:
C = Generate_Random_Schedule(teacherslist,courseslist)
print(Fitness(C))
C = mutate_child(C)
print(Fitness(C))

(72.5, False)
(71.0, False)


## Week Re-Assigner

In [50]:
def week_re_assign(L):

    weekds = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri',
              'Mon2', 'Tue2', 'Wed2', 'Thu2', 'Fri2',
              'Mon3', 'Tue3', 'Wed3', 'Thu3', 'Fri3']

    newl = [all for all in L]

    for i in range(len(newl)):
        newl[i] = (weekds[int(newl[i][0])], newl[i][1])

    return newl

# Genetic Algorithm

In [51]:
# Set number of days
weekDays = np.arange(0, 9)

iter = 0

# POPULATION SIZE
populationSize = 50
available_seats_per_slot = 280


# vars
notFound = True
population = []


# percentage of population to re-allocate
priorityReAllocatePercentage = 10


# number of exams to mutate
Mutation_percentage = 10
# less than 5 is nothing
# 5% = one change
# 10% = twocahnges

Mutation_count = int(len(courseslist) * Mutation_percentage/100)
Mutation_count

2

In [52]:
#Generate Initial Random Populations
population = generate_random_population(size = populationSize, seats=available_seats_per_slot )
population = Fitness_to_dictionary(population)
population

[{'Schedule': [{'Course ': 'DS3011', 'Day': 0, 'Slot': 0, 'Teachr': 'Adnan Tariq'},
   {'Course ': 'CS218', 'Day': 0, 'Slot': 0, 'Teachr': 'Muhammad Usman'},
   {'Course ': 'MT224', 'Day': 0, 'Slot': 0, 'Teachr': 'Sara Aziz'},
   {'Course ': 'CS328', 'Day': 0, 'Slot': 0, 'Teachr': 'Muhammad Usman'},
   {'Course ': 'SS118', 'Day': 0, 'Slot': 0, 'Teachr': 'Tayyab Nadeem'},
   {'Course ': 'CS211', 'Day': 0, 'Slot': 0, 'Teachr': 'Sara Aziz'},
   {'Course ': 'SS111', 'Day': 0, 'Slot': 0, 'Teachr': 'Javaria Imtiaz'},
   {'Course ': 'SE110', 'Day': 0, 'Slot': 0, 'Teachr': 'Sara Aziz'},
   {'Course ': 'MG220', 'Day': 0, 'Slot': 1, 'Teachr': 'Adnan Tariq'},
   {'Course ': 'CY2012', 'Day': 0, 'Slot': 1, 'Teachr': 'Zohaib Iqbal'},
   {'Course ': 'SS152', 'Day': 0, 'Slot': 1, 'Teachr': 'Adnan Tariq'},
   {'Course ': 'CS217', 'Day': 0, 'Slot': 1, 'Teachr': 'Farwa Batool'},
   {'Course ': 'CS118', 'Day': 0, 'Slot': 1, 'Teachr': 'Javaria Imtiaz'},
   {'Course ': 'CS302', 'Day': 0, 'Slot': 1, 'Teachr'

In [53]:
# Main Loop
print("Generated Randoms")

while notFound:
    iter += 1

    # sort the population in ascending order of fitness
    population = list(sorted(population, key=lambda x: x['fitness']))

    print("iter : ", iter, [x['fitness'] for x in population[:10]])

    # Ideal scenario found (without clashes, break the loop)
    if population[0]['solved']:

        checkHardConstraint(population[0]["Schedule"])
        print("ANSWER FOUND !!!!")
        # print(population[0]['Schedule'])

        break
        notFound = False

    # Else, Generated Children Fow New Generation
    newerGeneration = []

    # Selecting Top (priorityReAllocatePercentage) From Population with Highest Finess
    index = int((priorityReAllocatePercentage * populationSize)/100)

    newerGeneration.extend(population[:index])

    Offspring_Size = populationSize - index

    # produces new ofspring schedules

    #newborns = generate_offsprings(Offspring_Size,population[:index])
    newborns = generate_offsprings_with_roulette(
        Offspring_Size, population[:2*index])

    
    # Mutate new borns
    # converts new schedules to fitness dict and appends
    newerGeneration.extend(Fitness_to_dictionary(
        [mutate_child(newborn, changeby=Mutation_count) for newborn in newborns]))

    population = [x for x in newerGeneration]

Generated Randoms
iter :  1 [531.5, 541.0, 545.5, 547.0, 548.5, 549.0, 549.5, 551.5, 552.0, 552.0]
iter :  2 [292, 293, 295, 297, 303, 310, 311, 312, 316, 317]
iter :  3 [203, 212, 222, 223, 224.5, 226, 227, 231, 233, 238]
iter :  4 [162, 173, 173, 176.0, 176, 177, 178, 181, 181, 182]
iter :  5 [111, 114, 116, 131, 138, 140, 145.0, 146, 149, 150]
iter :  6 [77, 87, 92, 94, 100, 104.0, 104, 107, 107, 107]
iter :  7 [70.5, 71, 73, 74, 75, 75, 77, 78, 81.5, 82]
iter :  8 [54.0, 54.5, 55, 55, 57.5, 62, 65, 65.5, 65.5, 67.5]
iter :  9 [38.0, 49.5, 50.0, 50.5, 51.5, 52.0, 53, 53, 53.5, 54.0]
iter :  10 [31.5, 38.0, 39, 39.5, 42.0, 43, 45.0, 45.0, 46.0, 46.5]
iter :  11 [30.0, 31.5, 31.5, 31.5, 33.5, 34.0, 37.0, 37.5, 37.5, 37.5]
iter :  12 [19.5, 27.5, 28.5, 29.5, 30.0, 30.0, 30.5, 30.5, 31.5, 31.5]
iter :  13 [18.5, 19.5, 20.0, 20.5, 23.5, 23.5, 24.5, 25.5, 26.5, 26.5]
iter :  14 [18.5, 18.5, 19, 19.5, 20.0, 20.5, 22.0, 23.5, 24.5, 25.0]
iter :  15 [14.5, 17.5, 18.5, 18.5, 19, 19.0, 19, 19.

iter :  167 [2, 2, 2, 2, 2, 4.5, 5, 5, 7, 8]
iter :  168 [2, 2, 2, 2, 2, 2, 2, 2, 4, 5]
iter :  169 [2, 2, 2, 2, 2, 4, 5, 5, 6, 6]
iter :  170 [2, 2, 2, 2, 2, 3, 6.0, 7, 7, 7.0]
iter :  171 [2, 2, 2, 2, 2, 4, 5, 5, 7, 7.5]
iter :  172 [2, 2, 2, 2, 2, 4, 4, 7, 7, 7]
iter :  173 [2, 2, 2, 2, 2, 2, 3, 3, 3, 4]
iter :  174 [2, 2, 2, 2, 2, 5, 6.5, 6.5, 9, 10]
iter :  175 [2, 2, 2, 2, 2, 2, 6.5, 7.5, 8, 8]
iter :  176 [2, 2, 2, 2, 2, 2, 4.5, 5.5, 6, 7.5]
iter :  177 [2, 2, 2, 2, 2, 4, 6, 6.5, 7, 7.0]
iter :  178 [2, 2, 2, 2, 2, 2, 3, 4, 5, 6]
iter :  179 [2, 2, 2, 2, 2, 4, 5.5, 6, 6, 7]
iter :  180 [2, 2, 2, 2, 2, 5, 7, 8, 8, 9]
iter :  181 [2, 2, 2, 2, 2, 3, 3.0, 5, 5, 6.0]
iter :  182 [2, 2, 2, 2, 2, 4, 6, 6, 6, 6]
iter :  183 [2, 2, 2, 2, 2, 2.5, 7, 7.5, 8, 9]
iter :  184 [2, 2, 2, 2, 2, 2, 3, 4.0, 5, 6]
iter :  185 [2, 2, 2, 2, 2, 6, 7, 7.0, 8, 8]
iter :  186 [2, 2, 2, 2, 2, 6, 6, 7, 8, 8]
iter :  187 [2, 2, 2, 2, 2, 4.5, 5, 5, 6, 6]
iter :  188 [2, 2, 2, 2, 2, 7, 7.5, 9.5, 10, 11.0]
ite

## Recover after Stoping

In [54]:
population = [x for x in newerGeneration]

## Displaying Data

### Display as Table 

In [58]:
def Table(popval):
    d = [all.to_dict() for all in popval["Schedule"]]

    df = pd.DataFrame(d)

    table = df.groupby(by=['Day']).apply(lambda a: a[:])

    columns = ['Slot', 'Course ', 'Teachr']
    t1 = pd.DataFrame(table, columns=columns)

    t1.index = pd.MultiIndex.from_tuples(
        week_re_assign(t1.index), names=('Day', None))

    t1['Slot'] = t1['Slot'].map({0: '9am - 12pm', 1: '2pm - 5pm'})

    return t1

Table(population[0])

Slot Course           Teachr
Day                                        
Mon  0    2pm - 5pm   MG223       Sara Aziz
Tue  1   9am - 12pm   MG220   Tayyab Nadeem
     2    2pm - 5pm   MT224     Adnan Tariq
Wed  3   9am - 12pm   CS302   Tayyab Nadeem
     4   9am - 12pm   CS220  Javaria Imtiaz
     5    2pm - 5pm   SS111       Sara Aziz
Thu  6   9am - 12pm   SE110  Muhammad Usman
     7   9am - 12pm   EE227     Adnan Tariq
     8    2pm - 5pm   EE229    Farwa Batool
Fri  9   9am - 12pm  CY2012    Zohaib Iqbal
     10   2pm - 5pm  DS3011   Maheen Arshad
Mon2 11  9am - 12pm   CS218    Bilal Khalid
     12   2pm - 5pm   CS217  Javaria Imtiaz
     13   2pm - 5pm   SS113    Bilal Khalid
Tue2 14  9am - 12pm   CS211   Nagina Safdar
     15   2pm - 5pm   SS152   Tayyab Nadeem
     16   2pm - 5pm   CS118    Bilal Khalid
Wed2 17  9am - 12pm  AI2011     Adnan Tariq
     18  9am - 12pm   SS118  Javaria Imtiaz
     19   2pm - 5pm   CS307   Nagina Safdar
Thu2 20  9am - 12pm   CS328    Bilal Khalid
     21   2pm - 5pm   CS219     Adnan Tariq
     22   2pm - 5pm   MT205   Maheen Arshad

In [56]:
#Print Current Clashes
Fitness_with_print(population[0]["Schedule"])

Hard Clashes :  SE110 EE227 (3, 0) ['Tina Nasersharif']
Soft Clash :  DS3011 CY2012 ['Ali M Sadeghioon']


(1.5, False)

In [57]:
#Print Current Clashes
for i in range(10):
    print(i)    
    Fitness_with_print(population[i]["Schedule"])
    print("---------------")

0
Hard Clashes :  SE110 EE227 (3, 0) ['Tina Nasersharif']
Soft Clash :  DS3011 CY2012 ['Ali M Sadeghioon']
---------------
1
Hard Clashes :  EE227 SE110 (3, 0) ['Tina Nasersharif']
1 MG Courses Exams were Taken After CS Exams
---------------
2
Hard Clashes :  SE110 EE227 (3, 0) ['Tina Nasersharif']
1 MG Courses Exams were Taken After CS Exams
---------------
3
Hard Clashes :  SE110 EE227 (3, 0) ['Tina Nasersharif']
1 MG Courses Exams were Taken After CS Exams
---------------
4
Hard Clashes :  EE227 SE110 (3, 0) ['Tina Nasersharif']
1 MG Courses Exams were Taken After CS Exams
---------------
5
Hard Clashes :  SS152 DS3011 (6, 1) ['Hanna I Siahaan', 'Mohammed A Suleman', 'Ayda Niazi']
Hard Clashes :  CS118 DS3011 (6, 1) ['Adam C Ryan', 'Sam J Pottier', 'Sheila Hughton', 'Sarah French']
Hard Clashes :  EE227 SE110 (3, 0) ['Tina Nasersharif']
Hard Clashes :  MG220 EE229 (1, 0) ['Bibi A Khan', 'Anita Hodler', 'Sonia V Diaz', 'Saleem Abubacker', 'Mohammed Sukkari', 'Kamal Anwar', 'Ana B Jav